In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from fastai.callback.fp16 import to_fp16
from fastai.vision import *
from fastai.data.all import *
from fastai.vision.all import *

# function definitions

In [ ]:
def dl_accuracy(learner, dl, gpu=False):
    act, true_labels = learner.get_preds(dl=dl)
    preds = torch.argmax(act, axis=1)
    acc = torch.where(true_labels == preds, 1., 0.).mean()
    return acc.item()

In [ ]:
def nosplit(o): return L(int(i) for i in range(len(o))), L()

# load data

In [ ]:
if torch.cuda.is_available():
    use_gpu = True
    gpu = torch.device('cuda')
    print(torch.cuda.get_device_name(0))
else:
    print('No GPU available')

In [ ]:
train_dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                         get_items=get_image_files, 
                         splitter=RandomSplitter(seed=42),
                         get_y=parent_label,
                         item_tfms=Resize(460),
                         batch_tfms=[*aug_transforms(size=224, min_scale=0.75),
                                     Normalize.from_stats(*imagenet_stats)])

In [ ]:
test_dblock = DataBlock(blocks = (ImageBlock, CategoryBlock),
                        get_items=get_image_files, 
                        splitter=nosplit,
                        get_y=parent_label,
                        item_tfms=Resize(460),
                        batch_tfms=[*aug_transforms(size=224, min_scale=0.75),
                                    Normalize.from_stats(*imagenet_stats)])

In [ ]:
train_dls = train_dblock.dataloaders('mini_birds_dataset/train/', bs=256)
test_dls = test_dblock.dataloaders('mini_birds_dataset/test/', bs=128)

## Resnet34

In [ ]:
resnet_34_learn = vision_learner(train_dls, resnet34, metrics=accuracy, loss_func=nn.CrossEntropyLoss()).to_fp16()

In [ ]:
lr_min,lr_steep = resnet_34_learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
resnet_34_learn.fine_tune(15, freeze_epochs=3, base_lr=5e-2)

In [ ]:
dl_accuracy(resnet_34_learn, test_dls.train)

## XResnet34

In [ ]:
xresnet_34_learn = vision_learner(train_dls, xresnet34, metrics=accuracy, loss_func=nn.CrossEntropyLoss()).to_fp16()

In [ ]:
lr_min,lr_steep = xresnet_34_learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
xresnet_34_learn.fine_tune(15, freeze_epochs=3, base_lr=1.5e-1)

In [ ]:
dl_accuracy(xresnet_34_learn, test_dls.train)

## Resnet 50

In [ ]:
resnet_50_learn = vision_learner(train_dls, resnet50, metrics=accuracy, loss_func=nn.CrossEntropyLoss()).to_fp16()

In [ ]:
lr_min,lr_steep = resnet_50_learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
resnet_50_learn.fine_tune(15, freeze_epochs=3, base_lr=3e-3)

In [ ]:
dl_accuracy(resnet_50_learn, test_dls.train)

## Xresnet50

In [ ]:
xresnet_50_learn = vision_learner(train_dls, xresnet50, metrics=accuracy, loss_func=nn.CrossEntropyLoss()).to_fp16()

In [ ]:
lr_min,lr_steep = xresnet_50_learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
xresnet_50_learn.fine_tune(15, freeze_epochs=3, base_lr=9e-2)

In [ ]:
dl_accuracy(xresnet_50_learn, test_dls.train)